In [ ]:
import os
import docx2txt
from docx import Document
from docx.enum.text import WD_PARAGRAPH_ALIGNMENT
from docx.shared import Inches, Pt
from docx.enum.section import WD_SECTION

def reorganizar_imagenes_en_documento(nombre_documento, ruta_salida):
    # Crear carpeta temporal para las imágenes
    ruta_temporal = "imagenes_temp"
    if not os.path.exists(ruta_temporal):
        os.makedirs(ruta_temporal)
    
    # Extraer imágenes
    docx2txt.process(nombre_documento, ruta_temporal)
    
    # Obtener lista de imágenes en orden
    imagenes = []
    for i in range(1, 1000):
        for ext in ['png', 'jpg', 'jpeg']:
            nombre_imagen = f"image{i}.{ext}"
            ruta_imagen = os.path.join(ruta_temporal, nombre_imagen)
            if os.path.exists(ruta_imagen):
                imagenes.append(ruta_imagen)
                break
    
    # Crear nuevo documento
    nuevo_documento = Document()
    
    # Configurar el documento y reducir márgenes
    section = nuevo_documento.sections[0]
    section.page_height = Inches(11)
    section.page_width = Inches(8.5)
    # Reducir márgenes
    section.left_margin = Inches(0.5)
    section.right_margin = Inches(0.5)
    section.top_margin = Inches(0.5)
    section.bottom_margin = Inches(0.5)
    
    # Procesar las imágenes en pares
    for i in range(0, len(imagenes), 2):
        # Primera imagen
        parrafo1 = nuevo_documento.add_paragraph()
        parrafo1.alignment = WD_PARAGRAPH_ALIGNMENT.CENTER
        run1 = parrafo1.add_run()
        run1.add_picture(imagenes[i], width=Inches(6.0))  # Aumentado por márgenes reducidos
        
        # Añadir espacio entre imágenes
        espacio = nuevo_documento.add_paragraph()
        espacio.space_after = Pt(20)
        
        # Segunda imagen (si existe)
        if i + 1 < len(imagenes):
            parrafo2 = nuevo_documento.add_paragraph()
            parrafo2.alignment = WD_PARAGRAPH_ALIGNMENT.CENTER
            run2 = parrafo2.add_run()
            run2.add_picture(imagenes[i + 1], width=Inches(6.0))
        
        # Añadir salto de página si no es la última página
        if i + 2 < len(imagenes):
            nuevo_documento.add_page_break()
    
    # Crear nombre del nuevo documento
    nombre_base = os.path.splitext(os.path.basename(nombre_documento))[0]
    nuevo_nombre = os.path.join(ruta_salida, f"{nombre_base}_reorganizado.docx")
    
    # Guardar el nuevo documento
    nuevo_documento.save(nuevo_nombre)
    
    # Limpiar la carpeta temporal
    for img_file in os.listdir(ruta_temporal):
        os.remove(os.path.join(ruta_temporal, img_file))
    os.rmdir(ruta_temporal)

def procesar_todos_los_documentos():
    # Crear carpeta para documentos organizados si no existe
    ruta_salida = "documentos_organizados"
    if not os.path.exists(ruta_salida):
        os.makedirs(ruta_salida)
    
    # Obtener todos los documentos Word en la carpeta actual
    documentos = [f for f in os.listdir() if f.endswith('.docx') and not f.endswith('_reorganizado.docx')]
    
    print(f"Se encontraron {len(documentos)} documentos para procesar.")
    
    # Procesar cada documento
    for doc in documentos:
        try:
            print(f"Procesando: {doc}")
            reorganizar_imagenes_en_documento(doc, ruta_salida)
            print(f"✓ Completado: {doc}")
        except Exception as e:
            print(f"Error procesando {doc}: {str(e)}")
    
    print("\nProceso completado. Los documentos organizados se encuentran en la carpeta 'documentos_organizados'")

# Ejecutar el procesamiento de todos los documentos
if __name__ == "__main__":
    procesar_todos_los_documentos()